Debes crear las siguientes funciones para los endpoints que se consumirán en la API, recuerden que deben tener un decorador por cada una (@app.get(‘/’)).

def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

def UsersNotRecommend( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

In [1]:
import pandas as pd
import numpy as np
import ast
import gzip
import json
from pandas import json_normalize
from textblob import TextBlob


import pyarrow as pa
import pyarrow.parquet as pq


In [2]:

'''Funcion para extraer datos de un archivo JSON
    Lee las lineas JSON de un archivo gzippeado, 
    y lo transforma en un dataframe de pandas.
'''
def de_json_gz_a_df(file_path):
    with gzip.open(file_path, 'rt') as f:
        data = [json.loads(line) for line in f]
    return pd.DataFrame(data)

#Aplico la funcion para el archivo especifico y lo almaceno en un dataframe
df_steam_games = de_json_gz_a_df('steam_games.json.gz') 

In [177]:
'''Funcion para extraer datos de un archivo JSON
    Lee las lineas JSON de un archivo, 
    y lo transforma en un dataframe de pandas.
'''
def de_json_a_df(archivo):
    lista = []
    with open(archivo, encoding='utf-8') as file:
        for line in file.readlines():
            lista.append(ast.literal_eval(line))
    return pd.DataFrame(lista)

# Aplico la funcion para los archivos especificos y lo almaceno en un dataframe
df_user_reviews = de_json_a_df('user_reviews.json') 
df_user_items = de_json_a_df('users_items.json') 

In [3]:
#Creamos una funcion para mostrar los tipos de columnas, cantidad de filas nulas y % de nulos
def resumen(df):
    print(f'data shape: {df.shape}')
    resumen = pd.DataFrame(df.dtypes, columns=['data type'])
    resumen['#missing'] = df.isnull().sum()
    resumen['%missing'] = df.isnull().sum()*100/len(df)
    #resumen['#unique'] = df.nunique()
    return resumen

In [4]:
# Guarda el valor actual de 'display.max_colwidth'
original_max_colwidth = pd.get_option('display.max_colwidth')

# Establece el nuevo valor para 'display.max_colwidth'
pd.set_option('display.max_colwidth', None)

Arrancamos con la exploracion y depuracion de Steam_games

In [5]:
df_steam_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
print(resumen(df_steam_games))

#Se verifica un volumen elevado de nulos

data shape: (120445, 13)
             data type  #missing   %missing
publisher       object     96362  80.004982
genres          object     91593  76.045498
app_name        object     88312  73.321433
title           object     90360  75.021794
url             object     88310  73.319773
release_date    object     90377  75.035909
tags            object     88473  73.455104
reviews_url     object     88312  73.321433
specs           object     88980  73.876043
price           object     89687  74.463033
early_access    object     88310  73.319773
id              object     88312  73.321433
developer       object     91609  76.058782


In [7]:
# Contamos las filas donde todas las columnas son nulas
filas_nulas = df_steam_games[df_steam_games.isna().all(axis=1)]

# Obtenemos el número de filas
print(len(filas_nulas))

88310


In [8]:
# Utilizamos dropna how='all' para eliminar filas donde TODAS las columnas son nulas
df_steam_games_depurado = df_steam_games.dropna(how='all')

# Verificamos el DataFrame resultante
df_steam_games_depurado

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_Summoner_Kitty/,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?browsefilter=mostrecent&p=1,[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironbound/,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game, Trading Card Game, Turn-Based, Fantasy, Tactical, Dark Fantasy, Board Game, PvP, 2D, Competitive, Replay Value, Character Customization, Female Protagonist, Difficult, Design & Illustration]",http://steamcommunity.com/app/643980/reviews/?browsefilter=mostrecent&p=1,"[Single-player, Multi-player, Online Multi-Player, Cross-Platform Multiplayer, Steam Achievements, Steam Trading Cards, In-App Purchases]",Free To Play,False,643980,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_Pool_3D__Poolians/,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Indie, Multiplayer]",http://steamcommunity.com/app/670290/reviews/?browsefilter=mostrecent&p=1,"[Single-player, Multi-player, Online Multi-Player, In-App Purchases, Stats]",Free to Play,False,670290,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?browsefilter=mostrecent&p=1,[Single-player],0.99,False,767400,彼岸领域
88314,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_Challenge/,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?browsefilter=mostrecent&p=1,"[Single-player, Full controller support, HTC Vive, Oculus Rift, Tracked Motion Controllers, Room-Scale]",2.99,False,773570,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colony_On_Mars/,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?browsefilter=mostrecent&p=1,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGistICAL_South_Africa/,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?browsefilter=mostrecent&p=1,"[Single-player, Steam Achievements, Steam Cloud, Stats, Steam Leaderboards]",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russian_Roads/,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?browsefilter=mostrecent&p=1,"[Single-player, Steam Achievements, Steam Trading Cards]",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_2__Directions/,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmospheric, Relaxing]",http://steamcommunity.com/app/658870/reviews/?browsefilter=mostrecent&p=1,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [9]:
#Volvemos a correr el mismo analisis de nulos
print(resumen(df_steam_games_depurado))
print(df_steam_games_depurado.shape)

data shape: (32135, 13)
             data type  #missing   %missing
publisher       object      8052  25.056792
genres          object      3283  10.216275
app_name        object         2   0.006224
title           object      2050   6.379337
url             object         0   0.000000
release_date    object      2067   6.432239
tags            object       163   0.507235
reviews_url     object         2   0.006224
specs           object       670   2.084954
price           object      1377   4.285047
early_access    object         0   0.000000
id              object         2   0.006224
developer       object      3299  10.266065
(32135, 13)


In [10]:
#Analizamos la columna genres ya que sera uno de los datos mas relevantes para las funciones de la API,
# verificando los valores unicos
unique_genres = pd.unique(df_steam_games_depurado['genres'].explode())

print(unique_genres.shape)
print(type(unique_genres))
unique_genres


(23,)
<class 'numpy.ndarray'>


array(['Action', 'Casual', 'Indie', 'Simulation', 'Strategy',
       'Free to Play', 'RPG', 'Sports', 'Adventure', nan, 'Racing',
       'Early Access', 'Massively Multiplayer',
       'Animation &amp; Modeling', 'Video Production', 'Utilities',
       'Web Publishing', 'Education', 'Software Training',
       'Design &amp; Illustration', 'Audio Production', 'Photo Editing',
       'Accounting'], dtype=object)

In [11]:
#Corregimos algunos valores con ampersan y eliminamos los nulos
unique_genres[13]='Animation & Modeling'
unique_genres[19]='Design & Illustration'
unique_genres2= np.delete(unique_genres, 9, axis=0)

In [12]:
unique_genres2

array(['Action', 'Casual', 'Indie', 'Simulation', 'Strategy',
       'Free to Play', 'RPG', 'Sports', 'Adventure', 'Racing',
       'Early Access', 'Massively Multiplayer', 'Animation & Modeling',
       'Video Production', 'Utilities', 'Web Publishing', 'Education',
       'Software Training', 'Design & Illustration', 'Audio Production',
       'Photo Editing', 'Accounting'], dtype=object)

In [13]:
#Analizo los valores unicos de la columna Tags, ya que aparentemente contiene valores comunes con Genres y puede utilizarse para rellenar nulos de Genres
unique_tags = pd.unique(df_steam_games_depurado['tags'].explode())
print(unique_tags.shape)
unique_tags

(340,)


array(['Strategy', 'Action', 'Indie', 'Casual', 'Simulation',
       'Free to Play', 'RPG', 'Card Game', 'Trading Card Game',
       'Turn-Based', 'Fantasy', 'Tactical', 'Dark Fantasy', 'Board Game',
       'PvP', '2D', 'Competitive', 'Replay Value',
       'Character Customization', 'Female Protagonist', 'Difficult',
       'Design & Illustration', 'Sports', 'Multiplayer', 'Adventure',
       'FPS', 'Shooter', 'Third-Person Shooter', 'Sniper', 'Third Person',
       'Racing', 'Early Access', 'Survival', 'Pixel Graphics', 'Cute',
       'Physics', 'Science', 'VR', 'Tutorial', 'Classic', 'Gore',
       "1990's", 'Singleplayer', 'Sci-fi', 'Aliens', 'First-Person',
       'Story Rich', 'Atmospheric', 'Silent Protagonist',
       'Great Soundtrack', 'Moddable', 'Linear', 'Retro', 'Funny',
       'Turn-Based Strategy', 'Platformer', 'Side Scroller',
       'Massively Multiplayer', 'Clicker', 'Gothic', 'Isometric',
       'Stealth', 'Mystery', 'Assassin', 'Comedy', 'Stylized', 'Co-op',
     

In [14]:

# Identifico por cada fila los valores de generos contenidos en tags

# Función para realizar la intersección teniendo en cuenta valores no listas
def intersect_with_unique_genres(tags):
    if isinstance(tags, list):
        return list(set(tags).intersection(unique_genres2))
    else:
        return []

# Creo la columna genres in tags, donde quedaran los generos que existan en los valores de tags, por cada fila
# Aplicar la función de intersección
df_steam_games_depurado['genres_in_tags'] = df_steam_games_depurado['tags'].apply(intersect_with_unique_genres)

# Muestra el DataFrame resultante
df_steam_games_depurado.head()

/var/folders/83/mc_56f2n2y7bfw5_shj4jwg80000gn/T/ipykernel_9987/3964581304.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_steam_games_depurado['genres_in_tags'] = df_steam_games_depurado['tags'].apply(intersect_with_unique_genres)


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,genres_in_tags
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_Summoner_Kitty/,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?browsefilter=mostrecent&p=1,[Single-player],4.99,False,761140,Kotoshiro,"[Indie, Action, Simulation, Strategy, Casual]"
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironbound/,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game, Trading Card Game, Turn-Based, Fantasy, Tactical, Dark Fantasy, Board Game, PvP, 2D, Competitive, Replay Value, Character Customization, Female Protagonist, Difficult, Design & Illustration]",http://steamcommunity.com/app/643980/reviews/?browsefilter=mostrecent&p=1,"[Single-player, Multi-player, Online Multi-Player, Cross-Platform Multiplayer, Steam Achievements, Steam Trading Cards, In-App Purchases]",Free To Play,False,643980,Secret Level SRL,"[Indie, Free to Play, RPG, Strategy, Design & Illustration]"
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_Pool_3D__Poolians/,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Indie, Multiplayer]",http://steamcommunity.com/app/670290/reviews/?browsefilter=mostrecent&p=1,"[Single-player, Multi-player, Online Multi-Player, In-App Purchases, Stats]",Free to Play,False,670290,Poolians.com,"[Indie, Sports, Free to Play, Simulation, Casual]"
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?browsefilter=mostrecent&p=1,[Single-player],0.99,False,767400,彼岸领域,"[Action, Adventure, Casual]"
88314,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_Challenge/,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?browsefilter=mostrecent&p=1,"[Single-player, Full controller support, HTC Vive, Oculus Rift, Tracked Motion Controllers, Room-Scale]",2.99,False,773570,NaN,"[Indie, Action, Sports, Casual]"


In [15]:
# Creamos una mascara para identificar las filas con nulos en 'genres'
condition = df_steam_games_depurado['genres'].isnull()

# Rellenar nulos con valores de 'genres_in_tags'
df_steam_games_depurado.loc[condition, 'genres'] = df_steam_games_depurado.loc[condition, 'genres_in_tags'].apply(lambda x: x if isinstance(x, list) else [])


In [16]:
#Volvemos a verificar el resumen de df_steam_games_depurado
resumen(df_steam_games_depurado)

#Ya verificamos que genres esta sin nulos

data shape: (32135, 14)


,data type,#missing,%missing
publisher,object,8052,25.056792
genres,object,0,0.000000
app_name,object,2,0.006224
title,object,2050,6.379337
url,object,0,0.000000
release_date,object,2067,6.432239
tags,object,163,0.507235
reviews_url,object,2,0.006224
specs,object,670,2.084954
price,object,1377,4.285047


In [17]:
# Volvemos a verificar los valores unicos
unique_genres = pd.unique(df_steam_games_depurado['genres'].explode())

print(unique_genres.shape)
print(type(unique_genres))
unique_genres


(25,)
<class 'numpy.ndarray'>


array(['Action', 'Casual', 'Indie', 'Simulation', 'Strategy',
       'Free to Play', 'RPG', 'Sports', 'Adventure', 'Racing',
       'Early Access', 'Design & Illustration', 'Massively Multiplayer',
       'Education', nan, 'Animation &amp; Modeling', 'Video Production',
       'Utilities', 'Web Publishing', 'Software Training',
       'Design &amp; Illustration', 'Audio Production', 'Photo Editing',
       'Animation & Modeling', 'Accounting'], dtype=object)

In [18]:
#Realizamos algunas correcciones, nuevamente

# Reemplazar 'Animation &amp; Modeling' con 'Animation & Modeling'
df_steam_games_depurado['genres'] = df_steam_games_depurado['genres'].apply(lambda x: [genre.replace('&amp;', '&') if isinstance(genre, str) else genre for genre in x])

# Reemplazar 'Design &amp; Illustration' con 'Design & Illustration'
df_steam_games_depurado['genres'] = df_steam_games_depurado['genres'].apply(lambda x: [genre.replace('&amp;', '&') if isinstance(genre, str) else genre for genre in x])


/var/folders/83/mc_56f2n2y7bfw5_shj4jwg80000gn/T/ipykernel_9987/3515137457.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_steam_games_depurado['genres'] = df_steam_games_depurado['genres'].apply(lambda x: [genre.replace('&amp;', '&') if isinstance(genre, str) else genre for genre in x])
/var/folders/83/mc_56f2n2y7bfw5_shj4jwg80000gn/T/ipykernel_9987/3515137457.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_steam_games_depurado['genres'] = df_steam_games_depurado['genres'].apply(lambda x: [ge

In [19]:
# Volvemos a generar los valores unicos de genres, luego de la depuracion
unique_genres = pd.unique(df_steam_games_depurado['genres'].explode())

print(unique_genres.shape)
print(type(unique_genres))
unique_genres

(23,)
<class 'numpy.ndarray'>


array(['Action', 'Casual', 'Indie', 'Simulation', 'Strategy',
       'Free to Play', 'RPG', 'Sports', 'Adventure', 'Racing',
       'Early Access', 'Design & Illustration', 'Massively Multiplayer',
       'Education', nan, 'Animation & Modeling', 'Video Production',
       'Utilities', 'Web Publishing', 'Software Training',
       'Audio Production', 'Photo Editing', 'Accounting'], dtype=object)

In [20]:
unique_genres= np.delete(unique_genres, 14, axis=0)

In [21]:
print(unique_genres.shape)
unique_genres

(22,)


array(['Action', 'Casual', 'Indie', 'Simulation', 'Strategy',
       'Free to Play', 'RPG', 'Sports', 'Adventure', 'Racing',
       'Early Access', 'Design & Illustration', 'Massively Multiplayer',
       'Education', 'Animation & Modeling', 'Video Production',
       'Utilities', 'Web Publishing', 'Software Training',
       'Audio Production', 'Photo Editing', 'Accounting'], dtype=object)

In [22]:
#Ahora generaremos una columna dummie (True/False) por cada genero

# Crea un dataframe vacio
genres_dummies= pd.DataFrame()

# Itera sobre cada valor unico de Genres 
for genres in unique_genres:
    # Crea la columna con valor true o false por fila del dataframe df_steam_games_depurado
    genres_dummies[genres] = df_steam_games_depurado['genres'].apply(lambda x: True if x == genres else False)

genres_dummies

,Action,Casual,Indie,Simulation,Strategy,Free to Play,RPG,Sports,Adventure,Racing,...,Massively Multiplayer,Education,Animation & Modeling,Video Production,Utilities,Web Publishing,Software Training,Audio Production,Photo Editing,Accounting
88310,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
88311,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
88312,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
88313,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
88314,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
120441,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
120442,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
120443,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [23]:
# Concateno las nuevas columnas dummies al DataFrame original
df_steam_games_depurado = pd.concat([df_steam_games_depurado, genres_dummies], axis=1)
df_steam_games_depurado.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,...,Massively Multiplayer,Education,Animation & Modeling,Video Production,Utilities,Web Publishing,Software Training,Audio Production,Photo Editing,Accounting
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_Summoner_Kitty/,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?browsefilter=mostrecent&p=1,[Single-player],4.99,...,False,False,False,False,False,False,False,False,False,False
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironbound/,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game, Trading Card Game, Turn-Based, Fantasy, Tactical, Dark Fantasy, Board Game, PvP, 2D, Competitive, Replay Value, Character Customization, Female Protagonist, Difficult, Design & Illustration]",http://steamcommunity.com/app/643980/reviews/?browsefilter=mostrecent&p=1,"[Single-player, Multi-player, Online Multi-Player, Cross-Platform Multiplayer, Steam Achievements, Steam Trading Cards, In-App Purchases]",Free To Play,...,False,False,False,False,False,False,False,False,False,False
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_Pool_3D__Poolians/,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Indie, Multiplayer]",http://steamcommunity.com/app/670290/reviews/?browsefilter=mostrecent&p=1,"[Single-player, Multi-player, Online Multi-Player, In-App Purchases, Stats]",Free to Play,...,False,False,False,False,False,False,False,False,False,False
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?browsefilter=mostrecent&p=1,[Single-player],0.99,...,False,False,False,False,False,False,False,False,False,False
88314,NaN,"[Indie, Action, Sports, Casual]",Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_Challenge/,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?browsefilter=mostrecent&p=1,"[Single-player, Full controller support, HTC Vive, Oculus Rift, Tracked Motion Controllers, Room-Scale]",2.99,...,False,False,False,False,False,False,False,False,False,False


In [24]:

# Verificar que los valores no nulos en las columnas "title" y "app_name" sean iguales
son_iguales_no_nulos = (df_steam_games_depurado['title'].notna() & df_steam_games_depurado['app_name'].notna() & (df_steam_games_depurado['title'] == df_steam_games_depurado['app_name'])).all()

# Imprimir el resultado
if son_iguales_no_nulos:
    print("Los valores en las columnas 'title' y 'app_name' son iguales para todas las filas.")
else:
    print("Los valores en las columnas 'title' y 'app_name' no son iguales para todas las filas.")

frecuencia_iguales = (df_steam_games_depurado['title'].notna() & df_steam_games_depurado['app_name'].notna() & (df_steam_games_depurado['title'] == df_steam_games_depurado['app_name'])).sum()
frecuencia_distintos = len(df_steam_games_depurado) - frecuencia_iguales  
frecuencia_distintos = len(df_steam_games_depurado) - frecuencia_iguales  

print("Porcentaje de valores iguales:", round((frecuencia_iguales/len(df_steam_games_depurado))*100,0))

Los valores en las columnas 'title' y 'app_name' no son iguales para todas las filas.
Porcentaje de valores iguales: 92.0


In [25]:
#Considerando que el porcentaje de valores iguales es elevado y no tenemos muchos valores nulos en title 
# rellenamos los valores nulos de title con los valores de la columna app_name

df_steam_games_depurado['title'] = df_steam_games_depurado['title'].fillna(df_steam_games_depurado['app_name'])

In [26]:
resumen(df_steam_games_depurado)

data shape: (32135, 36)


,data type,#missing,%missing
publisher,object,8052,25.056792
genres,object,0,0.000000
app_name,object,2,0.006224
title,object,2,0.006224
url,object,0,0.000000
release_date,object,2067,6.432239
tags,object,163,0.507235
reviews_url,object,2,0.006224
specs,object,670,2.084954
price,object,1377,4.285047


In [27]:
#Analizamos la columna specs verificando los valores unicos
unique_specs = pd.unique(df_steam_games_depurado['specs'].explode())

print(unique_specs.shape)
print(type(unique_specs))
unique_specs

#Teniendo en cuenta que la cantidad de valores unicos es considerablemente alta...
#no lo convertiremos en columnas dummies


(41,)
<class 'numpy.ndarray'>


array(['Single-player', 'Multi-player', 'Online Multi-Player',
       'Cross-Platform Multiplayer', 'Steam Achievements',
       'Steam Trading Cards', 'In-App Purchases', 'Stats',
       'Full controller support', 'HTC Vive', 'Oculus Rift',
       'Tracked Motion Controllers', 'Room-Scale', 'Downloadable Content',
       'Steam Cloud', 'Steam Leaderboards', 'Partial Controller Support',
       'Seated', 'Standing', 'Local Co-op', 'Shared/Split Screen', nan,
       'Valve Anti-Cheat enabled', 'Local Multi-Player',
       'Steam Turn Notifications', 'MMO', 'Co-op', 'Online Co-op',
       'Captions available', 'Commentary available', 'Steam Workshop',
       'Includes level editor', 'Mods', 'Mods (require HL2)', 'Game demo',
       'Includes Source SDK', 'SteamVR Collectibles', 'Keyboard / Mouse',
       'Gamepad', 'Windows Mixed Reality', 'Mods (require HL1)'],
      dtype=object)

In [28]:
#Creamos la columna anho de lanzamiento desde el dato release_date tomando solo el anho de aquellos datos con el formato correcto
df_steam_games_depurado['release_date'] = pd.to_datetime(df_steam_games_depurado['release_date'], errors='coerce', format='%Y-%m-%d')
df_steam_games_depurado['anho_lanzamiento'] = df_steam_games_depurado['release_date'].dt.year
df_steam_games_depurado['anho_lanzamiento'] = df_steam_games_depurado['anho_lanzamiento'].astype('Int64')
resumen(df_steam_games_depurado)
df_steam_games_depurado.head()

data shape: (32135, 37)


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,...,Education,Animation & Modeling,Video Production,Utilities,Web Publishing,Software Training,Audio Production,Photo Editing,Accounting,anho_lanzamiento
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_Summoner_Kitty/,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?browsefilter=mostrecent&p=1,[Single-player],4.99,...,False,False,False,False,False,False,False,False,False,2018
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironbound/,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game, Trading Card Game, Turn-Based, Fantasy, Tactical, Dark Fantasy, Board Game, PvP, 2D, Competitive, Replay Value, Character Customization, Female Protagonist, Difficult, Design & Illustration]",http://steamcommunity.com/app/643980/reviews/?browsefilter=mostrecent&p=1,"[Single-player, Multi-player, Online Multi-Player, Cross-Platform Multiplayer, Steam Achievements, Steam Trading Cards, In-App Purchases]",Free To Play,...,False,False,False,False,False,False,False,False,False,2018
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_Pool_3D__Poolians/,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Indie, Multiplayer]",http://steamcommunity.com/app/670290/reviews/?browsefilter=mostrecent&p=1,"[Single-player, Multi-player, Online Multi-Player, In-App Purchases, Stats]",Free to Play,...,False,False,False,False,False,False,False,False,False,2017
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?browsefilter=mostrecent&p=1,[Single-player],0.99,...,False,False,False,False,False,False,False,False,False,2017
88314,NaN,"[Indie, Action, Sports, Casual]",Log Challenge,Log Challenge,http://store.steampowered.com/app/773570/Log_Challenge/,NaT,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?browsefilter=mostrecent&p=1,"[Single-player, Full controller support, HTC Vive, Oculus Rift, Tracked Motion Controllers, Room-Scale]",2.99,...,False,False,False,False,False,False,False,False,False,<NA>


In [29]:
#Convertimos el tipo de datos de los datos numericos  

# Convierte la columna 'price' a tipo de dato float64
df_steam_games_depurado['price'] = df_steam_games_depurado['price'].replace('Free To Play', pd.NA)
df_steam_games_depurado['price'] = df_steam_games_depurado['price'].replace('Free to Play', pd.NA)
df_steam_games_depurado['price'] = pd.to_numeric(df_steam_games_depurado['price'], errors='coerce')
df_steam_games_depurado['price'] = df_steam_games_depurado['price'].astype('float64')

# Convierte la columna 'id' a tipo de dato float64
df_steam_games_depurado['id'] = pd.to_numeric(df_steam_games_depurado['id'], errors='coerce')


In [30]:
#Eliminamos las filas que no contengan el dato Id o Title (son pocas)
df_steam_games_depurado = df_steam_games_depurado.dropna(subset=['id', 'title'])

In [31]:
#Por ultimo, eliminamos las columnas no relevantes para las funciones que se crearan
columnas_a_eliminar = ['publisher', 'app_name', 'url', 'reviews_url', 'genres_in_tags', 'release_date', 'genres', 'early_access']
df_steam_games_depurado = df_steam_games_depurado.drop(columnas_a_eliminar, axis=1)
resumen(df_steam_games_depurado)


data shape: (32132, 29)


,data type,#missing,%missing
title,object,0,0.000000
tags,object,162,0.504170
specs,object,669,2.082037
price,float64,3286,10.226565
id,float64,0,0.000000
developer,object,3297,10.260799
Action,bool,0,0.000000
Casual,bool,0,0.000000
Indie,bool,0,0.000000
Simulation,bool,0,0.000000


In [32]:
df_steam_games_depurado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32132 entries, 88310 to 120444
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   title                  32132 non-null  object 
 1   tags                   31970 non-null  object 
 2   specs                  31463 non-null  object 
 3   price                  28846 non-null  float64
 4   id                     32132 non-null  float64
 5   developer              28835 non-null  object 
 6   Action                 32132 non-null  bool   
 7   Casual                 32132 non-null  bool   
 8   Indie                  32132 non-null  bool   
 9   Simulation             32132 non-null  bool   
 10  Strategy               32132 non-null  bool   
 11  Free to Play           32132 non-null  bool   
 12  RPG                    32132 non-null  bool   
 13  Sports                 32132 non-null  bool   
 14  Adventure              32132 non-null  bool   
 15  Ra

In [38]:
#Para identificar duplicados,  eliminamos las columnas que son listas y lo guardamos en un subdataframe, para poder aplicar el metodo duplicated
columnas_a_eliminar = ['tags', 'specs']
sub_df_steam_games_depurado = df_steam_games_depurado.drop(columnas_a_eliminar, axis=1)


In [39]:
#Verificamos si hay registros duplicados (todas las columnas con valores iguales)
duplicates = sub_df_steam_games_depurado[sub_df_steam_games_depurado.duplicated()]
duplicates.shape

(0, 27)

In [40]:
duplicates
#identificamos una sola fila duplicada

,title,price,id,developer,Action,Casual,Indie,Simulation,Strategy,Free to Play,...,Education,Animation & Modeling,Video Production,Utilities,Web Publishing,Software Training,Audio Production,Photo Editing,Accounting,anho_lanzamiento


In [37]:
#eliminamos la fila duplicada
df_steam_games_depurado = df_steam_games_depurado.drop(102883)

In [41]:
#Exportamos los datos depurados en formato parquet
df_steam_games_pya = pa.Table.from_pandas(df_steam_games_depurado)
pq.write_table(df_steam_games_pya, "df_steam_games.parquet")

Continuo con la depuracion de df_user_reviews

In [76]:
print(df_user_reviews.shape)
df_user_reviews.head()


(25799, 3)


,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970982479,"[{'funny': '', 'posted': 'Posted November 5, 2011.', 'last_edited': '', 'item_id': '1250', 'helpful': 'No ratings yet', 'recommend': True, 'review': 'Simple yet with great replayability. In my opinion does ""zombie"" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth ""zombie"" splattering fun for the whole family. Amazed this sort of FPS is so rare.'}, {'funny': '', 'posted': 'Posted July 15, 2011.', 'last_edited': '', 'item_id': '22200', 'helpful': 'No ratings yet', 'recommend': True, 'review': 'It's unique and worth a playthrough.'}, {'funny': '', 'posted': 'Posted April 21, 2011.', 'last_edited': '', 'item_id': '43110', 'helpful': 'No ratings yet', 'recommend': True, 'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]"
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014.', 'last_edited': '', 'item_id': '251610', 'helpful': '15 of 20 people (75%) found this review helpful', 'recommend': True, 'review': 'I know what you think when you see this title ""Barbie Dreamhouse Party"" but do not be intimidated by it's title, this is easily one of my GOTYs. You don't get any of that cliche game mechanics that all the latest games have, this is simply good core gameplay. Yes, you can't 360 noscope your friends, but what you can do is show them up with your bad ♥♥♥ dance moves and put them to shame as you show them what true fashion and color combinations are.I know this game says for kids but, this is easily for any age range and any age will have a blast playing this.8/8'}, {'funny': '', 'posted': 'Posted September 8, 2013.', 'last_edited': '', 'item_id': '227300', 'helpful': '0 of 1 people (0%) found this review helpful', 'recommend': True, 'review': 'For a simple (it's actually not all that simple but it can be!) truck driving Simulator, it is quite a fun and relaxing game. Playing on simple (or easy?) its just the basic WASD keys for driving but (if you want) the game can be much harder and realistic with having to manually change gears, much harder turning, etc. And reversing in this game is a ♥♥♥♥♥, as I imagine it would be with an actual truck. Luckily, you don't have to reverse park it but you get extra points if you do cause it is bloody hard. But this is suprisingly a nice truck driving game and I had a bit of fun with it.'}, {'funny': '', 'posted': 'Posted November 29, 2013.', 'last_edited': '', 'item_id': '239030', 'helpful': '1 of 4 people (25%) found this review helpful', 'recommend': True, 'review': 'Very fun little game to play when your bored or as a time passer. Very gud. Do Recommend. pls buy'}]"
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.', 'last_edited': '', 'item_id': '248820', 'helpful': 'No ratings yet', 'recommend': True, 'review': 'A suitably punishing roguelike platformer. Winning feels good. Progressive unlocks mean a good slog ending in failure doesn't feel like a waste.'}, {'funny': '', 'posted': 'Posted December 4, 2015.', 'last_edited': 'Last edited December 5, 2015.', 'item_id': '370360', 'helpful': 'No ratings yet', 'recommend': True, 'review': '""Run for fun? What the hell kind of fun is that?""'}, {'funny': '', 'posted': 'Posted November 3, 2014.', 'last_edited': '', 'item_id': '237930', 'helpful': 'No ratings yet', 'recommend': True, 'review': 'Elegant integration of gameplay, story, world development and aesthetic.'}, {'funny': '', 'posted': 'Posted October 15, 2014.', 'last_edited': '', 'item_id': '263360', 'helpful': 'No ratings yet', 'recommend': True, 'review': 'Random drops and random quests, with stat points. Animation style reminiscent of the era before the Voodoo card.'}, {'funny': '', 'posted': 'Posted O

In [77]:
#Creamos una columna num_elements para verificar cuantos reviews tiene cada fila
df_user_reviews['num_elements'] = df_user_reviews['reviews'].apply(len)
print(df_user_reviews['num_elements'].value_counts())
#Verificamos registros que no tienen reviews

num_elements
1     13665
2      4681
3      2561
4      1529
5      1004
6       746
7       563
8       425
9       328
10      269
0        28
Name: count, dtype: int64


In [113]:
#Eliminanos aquellos que no tienen reviews
filtro = df_user_reviews['num_elements'] == 0
# Eliminar las filas que cumplen con la condición
df_user_reviews_depurado = df_user_reviews.drop(df_user_reviews[filtro].index)

In [114]:
#Como reviews es un diccionario, guardamos solo esa columna para desempaquetarlo en varias columnas
solo_reviews = json_normalize(df_user_reviews_depurado['reviews'].explode(), sep='_').reset_index(drop=True)

print(solo_reviews.shape)
solo_reviews.head()

(59305, 7)


,funny,posted,last_edited,item_id,helpful,recommend,review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,"Simple yet with great replayability. In my opinion does ""zombie"" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth ""zombie"" splattering fun for the whole family. Amazed this sort of FPS is so rare."
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,"I know what you think when you see this title ""Barbie Dreamhouse Party"" but do not be intimidated by it's title, this is easily one of my GOTYs. You don't get any of that cliche game mechanics that all the latest games have, this is simply good core gameplay. Yes, you can't 360 noscope your friends, but what you can do is show them up with your bad ♥♥♥ dance moves and put them to shame as you show them what true fashion and color combinations are.I know this game says for kids but, this is easily for any age range and any age will have a blast playing this.8/8"
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,"For a simple (it's actually not all that simple but it can be!) truck driving Simulator, it is quite a fun and relaxing game. Playing on simple (or easy?) its just the basic WASD keys for driving but (if you want) the game can be much harder and realistic with having to manually change gears, much harder turning, etc. And reversing in this game is a ♥♥♥♥♥, as I imagine it would be with an actual truck. Luckily, you don't have to reverse park it but you get extra points if you do cause it is bloody hard. But this is suprisingly a nice truck driving game and I had a bit of fun with it."


In [115]:
# Concatenamos las columnas user_id y user_url con las columnas separadas de review
df_user_reviews_depurado = pd.concat([
    pd.DataFrame(np.repeat(df_user_reviews_depurado[['user_id', 'user_url']].values, df_user_reviews_depurado['reviews'].apply(len), axis=0), columns=['user_id', 'user_url']).reset_index(drop=True),
    solo_reviews[['funny', 'posted', 'last_edited', 'item_id', 'helpful', 'recommend', 'review']].reset_index(drop=True)
], axis=1) 

In [116]:
df_user_reviews_depurado.info()
df_user_reviews_depurado.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59305 non-null  object
 1   user_url     59305 non-null  object
 2   funny        59305 non-null  object
 3   posted       59305 non-null  object
 4   last_edited  59305 non-null  object
 5   item_id      59305 non-null  object
 6   helpful      59305 non-null  object
 7   recommend    59305 non-null  bool  
 8   review       59305 non-null  object
dtypes: bool(1), object(8)
memory usage: 3.7+ MB


(59305, 9)

In [117]:
#Verificamos si hay registros duplicados (todas las columnas con valores iguales)
duplicates = df_user_reviews_depurado[df_user_reviews_depurado.duplicated()]
duplicates.shape


(874, 9)

In [118]:
#Borramos los registros duplicados
df_user_reviews_depurado = df_user_reviews_depurado.drop_duplicates()
df_user_reviews_depurado.shape

(58431, 9)

In [119]:
# Definimos una función para realizar un análisis de sentimiento y aplicarla a la columna 'review'
def analisis_sentimiento(text):
    if text is None:
        return 1  # Valor neutro si la revisión es nula
    else:
        analysis = TextBlob(text)
        if analysis.sentiment.polarity > 0:
            return 2  # Positivo
        elif analysis.sentiment.polarity == 0:
            return 1  # Neutral
        else:
            return 0  # Negativo

# Aplicamos la función a la columna 'review' y creamos la nueva columna 'sentiment_analysis'
df_user_reviews_depurado['sentiment_analysis'] = df_user_reviews_depurado['review'].apply(analisis_sentimiento)


In [120]:
df_user_reviews_depurado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58431 entries, 0 to 59304
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             58431 non-null  object
 1   user_url            58431 non-null  object
 2   funny               58431 non-null  object
 3   posted              58431 non-null  object
 4   last_edited         58431 non-null  object
 5   item_id             58431 non-null  object
 6   helpful             58431 non-null  object
 7   recommend           58431 non-null  bool  
 8   review              58431 non-null  object
 9   sentiment_analysis  58431 non-null  int64 
dtypes: bool(1), int64(1), object(8)
memory usage: 4.5+ MB


In [124]:
# Extraemos el año como cadena de la columna 'posted'
df_user_reviews_depurado['anho_posteo'] = df_user_reviews_depurado['posted'].str.extract(r'(\d{4})')

# Convierte la columna 'year' a tipo de dato numérico (int)
df_user_reviews_depurado['anho_posteo'] = pd.to_numeric(df_user_reviews_depurado['anho_posteo'], errors='coerce')
df_user_reviews_depurado['anho_posteo'] = df_user_reviews_depurado['anho_posteo'].astype('Int64')
print(df_user_reviews_depurado['anho_posteo'].value_counts(dropna=False))

anho_posteo
2014    21834
2015    18154
<NA>     9933
2013     6713
2012     1201
2011      530
2010       66
Name: count, dtype: Int64


In [125]:
#Eliminamos las columnas no relevantes para las funciones que se crearan
columnas_a_eliminar = ['user_url', 'funny', 'posted', 'last_edited', 'helpful', 'review']
df_user_reviews_depurado = df_user_reviews_depurado.drop(columnas_a_eliminar, axis=1)

data shape: (58431, 5)


,data type,#missing,%missing
user_id,object,0,0.000000
item_id,object,0,0.000000
recommend,bool,0,0.000000
sentiment_analysis,int64,0,0.000000
anho_posteo,Int64,9933,16.999538


In [126]:
df_user_reviews_depurado['item_id'] = pd.to_numeric(df_user_reviews_depurado['item_id'], errors='coerce')
df_user_reviews_depurado['item_id'] = df_user_reviews_depurado['item_id'].astype('Int64')
resumen(df_user_reviews_depurado)

data shape: (58431, 5)


,data type,#missing,%missing
user_id,object,0,0.000000
item_id,Int64,0,0.000000
recommend,bool,0,0.000000
sentiment_analysis,int64,0,0.000000
anho_posteo,Int64,9933,16.999538


In [127]:
#Exportamos los datos depurados en formato parquet
df_reviews_pya = pa.Table.from_pandas(df_user_reviews_depurado)
pq.write_table(df_reviews_pya, "df_reviews.parquet")

Por ultimo, depuramos User Items

In [178]:
print(df_user_items.shape)
df_user_items.head()

(88310, 5)


user_id  items_count           steam_id  \
0  76561197970982479          277  76561197970982479   
1            js41637          888  76561198035864385   
2          evcentric          137  76561198007712555   
3         Riot-Punch          328  76561197963445855   
4              doctr          541  76561198002099482   

                                               user_url  \
0  http://steamcommunity.com/profiles/76561197970982479   
1                  http://steamcommunity.com/id/js41637   
2                http://steamcommunity.com/id/evcentric   
3               http://steamcommunity.com/id/Riot-Punch   
4                    http://steamcommunity.com/id/doctr   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [179]:
resumen(df_user_items)

data shape: (88310, 5)


,data type,#missing,%missing
user_id,object,0,0.0
items_count,int64,0,0.0
steam_id,object,0,0.0
user_url,object,0,0.0
items,object,0,0.0


In [180]:
df_user_items['items_count'].value_counts(dropna=False)
#Verificamos que existen registros sin items

items_count
0       16806
1        3304
2        1772
3        1521
4        1388
        ...  
1621        1
1103        1
942         1
618         1
1173        1
Name: count, Length: 925, dtype: int64

In [181]:
#Verificamos algunos ejemplos de registros sin items
df_user_items[df_user_items['items_count']==0]
#Estos registros no son utilies ya que no aportan ningun dato

,user_id,items_count,steam_id,user_url,items
9,Wackky,0,76561198039117046,http://steamcommunity.com/id/Wackky,[]
11,76561198079601835,0,76561198079601835,http://steamcommunity.com/profiles/76561198079601835,[]
31,hellom8o,0,76561198117222320,http://steamcommunity.com/id/hellom8o,[]
38,starkillershadow553,0,76561198059648579,http://steamcommunity.com/id/starkillershadow553,[]
54,darkenkane,0,76561198058876001,http://steamcommunity.com/id/darkenkane,[]
...,...,...,...,...,...
88298,76561198316380182,0,76561198316380182,http://steamcommunity.com/profiles/76561198316380182,[]
88299,76561198316970597,0,76561198316970597,http://steamcommunity.com/profiles/76561198316970597,[]
88300,76561198318100691,0,76561198318100691,http://steamcommunity.com/profiles/76561198318100691,[]
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClown77xX,[]


In [184]:
#Eliminamos las filas que no tienen items
filtro=df_user_items['items_count']==0

# Eliminar las filas que cumplen con la condición de filtro
df_user_items_depurado = df_user_items.drop(df_user_items[filtro].index)

In [185]:
df_user_items_depurado.shape

(71504, 5)

In [186]:
#Como la columna items es un diccionario, guardamos solo esa columna para desempaquetarlo en varias columnas
solo_items=json_normalize(df_user_items_depurado['items'].explode(), sep='_')
solo_items.head()

,item_id,item_name,playtime_forever,playtime_2weeks
0,10,Counter-Strike,6,0
1,20,Team Fortress Classic,0,0
2,30,Day of Defeat,7,0
3,40,Deathmatch Classic,0,0
4,50,Half-Life: Opposing Force,0,0


In [187]:
# Concatenamos las columnas distintas a Item con las columnas separadas de Item
df_user_items_depurado = pd.concat([
    pd.DataFrame(np.repeat(df_user_items_depurado[['user_id', 'steam_id', 'items_count', 'user_url']].values, df_user_items_depurado['items'].apply(len), axis=0), columns=['user_id', 'steam_id', 'items_count', 'user_url']).reset_index(drop=True),
    solo_items[['item_id', 'item_name', 'playtime_forever', 'playtime_2weeks']].reset_index(drop=True)
], axis=1) 


In [188]:
df_user_items_depurado.head()

,user_id,steam_id,items_count,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,76561197970982479,277,http://steamcommunity.com/profiles/76561197970982479,10,Counter-Strike,6,0
1,76561197970982479,76561197970982479,277,http://steamcommunity.com/profiles/76561197970982479,20,Team Fortress Classic,0,0
2,76561197970982479,76561197970982479,277,http://steamcommunity.com/profiles/76561197970982479,30,Day of Defeat,7,0
3,76561197970982479,76561197970982479,277,http://steamcommunity.com/profiles/76561197970982479,40,Deathmatch Classic,0,0
4,76561197970982479,76561197970982479,277,http://steamcommunity.com/profiles/76561197970982479,50,Half-Life: Opposing Force,0,0


In [189]:
resumen(df_user_items_depurado)

data shape: (5153209, 8)


,data type,#missing,%missing
user_id,object,0,0.0
steam_id,object,0,0.0
items_count,object,0,0.0
user_url,object,0,0.0
item_id,object,0,0.0
item_name,object,0,0.0
playtime_forever,int64,0,0.0
playtime_2weeks,int64,0,0.0


In [190]:
#Eliminamos las columnas no relevantes para las funciones que se crearan
columnas_a_eliminar = ['steam_id', 'items_count', 'user_url', 'item_name', 'playtime_2weeks']
df_user_items_depurado = df_user_items_depurado.drop(columnas_a_eliminar, axis=1)

In [191]:
resumen(df_user_items_depurado)

data shape: (5153209, 3)


,data type,#missing,%missing
user_id,object,0,0.0
item_id,object,0,0.0
playtime_forever,int64,0,0.0


In [192]:
df_user_items_depurado['item_id'] = pd.to_numeric(df_user_items_depurado['item_id'], errors='coerce')
df_user_items_depurado['item_id'] = df_user_items_depurado['item_id'].astype('Int64')
resumen(df_user_items_depurado)

In [193]:
resumen(df_user_items_depurado)

data shape: (5153209, 3)


,data type,#missing,%missing
user_id,object,0,0.0
item_id,Int64,0,0.0
playtime_forever,int64,0,0.0


In [194]:
#Verificamos si hay registros duplicados (todas las columnas con valores iguales)
duplicates = df_user_items_depurado[df_user_items_depurado.duplicated()]
duplicates.shape


(59117, 3)

In [195]:
#Borramos los registros duplicados
df_user_items_depurado = df_user_items_depurado.drop_duplicates()
df_user_items_depurado.shape

(5094092, 3)

In [196]:
#Exportamos los datos depurados en formato parquet
df_items_pya = pa.Table.from_pandas(df_user_items_depurado)
pq.write_table(df_items_pya, "df_items.parquet")